Note: The dataset used here was downloaded from https://www.gencodegenes.org/human/, under "Fasta files" and next to "Protein-coding transcript sequences".


In [ ]:
%config Completer.use_jedi = False
import pandas as pd
from Bio import SeqIO

filename = '../sequence-data/gencode.v48.pc_transcripts.fa'
sequences = SeqIO.parse(filename, 'fasta') # Use SeqIO.parse(filename, 'file type (usually fasta)') to parse files like fasta files

# This is just a quick way to get the first sequence in the file:
for record in sequences: # The parsed fasta file is an iterable object
    example = record 
    break

print(example)
type(example) # example stores a SeqRecord object

In [ ]:
print(example.id) # stores the contents of the ID record
print()
print(example.seq)
example.seq # example.seq stores the seq object

In [ ]:
# Note that the CDS in the example.id stands for the Coding DNA Sequence -- it shows the nucleotide start and end positions of the coding region of the sequence

In [ ]:
# The ID shows how the CDS for this sequence is 61-1041, so we can show this by indexing from 60 (to account for 0-based indexing) to 1041 (since Python slicing is exclusive) 
CDS = example.seq[60:1041]
print(CDS) # Prints only the coding region; we can confirm this by seeing the start codon (ATG for DNA, AUG for RNA) at the beginning of the printed sequence

In [ ]:
# seq_object.translate() translates the coding sequence into an amino acid sequence; the * at the end signifies a stop codon
print(CDS.translate())

In [ ]:
# This counts the number of sequences in the FASTA file
# Each sequence is parsed as a SeqRecord object, and sum(1 for _) adds 1 for each one found
sum(1 for _ in SeqIO.parse('../sequence-data/gencode.v48.pc_transcripts.fa', 'fasta'))

In [ ]:
# Confirmation/another way to count
list = []
sequences = SeqIO.parse('../sequence-data/gencode.v48.pc_transcripts.fa', 'fasta')
for record in sequences:
    list.append(record)
len(list)

In [ ]:
# Using a list is very helpful, as it permanently stores the sequence objects in a regular Python list that I can constantly access
print(list[0])
print()
print(list[0].id)
print()
print(list[0].seq)

## Pairwise Sequence Alignment

- Pairwise Sequence Alignment is used to identify regions of similarity that may indicate functional, structural, and/or evolutionary relationships between two biological sequences (protein or nucleic acid).

- Identifying the similar region enables us to infer a lot of information like what traits are conserved between species, how close different species genetically are, how species evolve, etc.

- Pairwise sequence alignment uses dynamic programming to the optimal alignment between the two sequences, scoring based on their similarity or distance (how different they are), and then assessing the significance of this score.

**Types of pairwise alignments:**
1. Global alignment: This method finds the best alignment over the entire lengths of the 2 sequences; what is the maximum similarity between sequence X and Y?
2. Local alignment: This method finds the most similar subsequences among the 2 sequences; What is the maximum similarity between a subsequence of X and a subsequence of Y?

**When doing alignments, you can specify the match score and gap penalties.**
1. The match score indicates the compatibility between an alignment of two characters in the sequences. Highly compatible characters should be given positive scores, and incompatible ones should be given negative scores or O.
2. The gap penalties should be negative.
   - The gap penalty subtracts from the score overall compatibility score whenever an insertion or deletion is involved in the alignment. This ensures that not too many gaps are introduced and acknowledges that sequences are still somewhat evolutionarily different whenever an insertion or deletion is accounted for to improve the match score.

Biopython offers two tools for pairwise sequence alignment:

**Bio.Align.PairwiseAligner** (modern, recommended)

Bio.pairwise2 (deprecated, older)



The **PairwiseAligner class** allows you to define how character matches and mismatches are scored using the following options:

1. Constant Match/Mismatch Scores:
Use simple numeric values for matches and mismatches:

<code>aligner.match_score = 1</code> # Score for identical characters
<br>
<code>aligner.mismatch_score = -1</code> # Penalty for different characters
<br>

2. Substitution Matrix:
For amino acid or nucleotide alignments with biologically relevant scores (e.g. BLOSUM62 or NUC.4.4), use a substitution matrix:

<code>from Bio.Align.substitution_matrices import load
aligner.substitution_matrix = load("BLOSUM62")</code>
<br>
Note: If a substitution matrix is set, match_score and mismatch_score are ignored because the matrix already has established rules that account for matches and mismatches.

3. Custom Scoring via Dictionary:
You can manually define a dictionary that maps character pairs to scores and convert it into a matrix:

<code>from Bio.Align import substitution_matrices
custom_dict = {('A', 'A'): 2, ('A', 'T'): -1, ('T', 'T'): 2}
matrix = substitution_matrices.Array(data=custom_dict)
aligner.substitution_matrix = matrix</code>
<br>

4. Fine-Grained Control (Advanced):
You can also subclass or modify the scoring behavior via advanced matrix construction or callbacks, though this is less common in PairwiseAligner.

The PairwiseAligner class allows you to define how gap penalties are handled using the following options:

1. Constant Open/Extend Gap Penalties:
Set the penalty for opening a gap and extending a gap:

<code>aligner.open_gap_score = -2</code> # Penalty for starting a new gap
<br>
<code>aligner.extend_gap_score = -0.5</code> # Penalty for extending an existing gap
<br>

2. Asymmetric Gap Penalties (Advanced):
You can set different penalties for gaps in sequence A vs sequence B:

<code>aligner.target_open_gap_score = -2
aligner.target_extend_gap_score = -0.5
aligner.query_open_gap_score = -1.5
aligner.query_extend_gap_score = -0.2</code>
<br>
Note: target is the first sequence, query is the second: <code>aligner.align(target, query)</code>

3. Callback Functions (Rarely Used):
You can define custom gap penalty functions, though this is advanced and rarely necessary.

Notes:
- Open gap penalty: A larger penalty for introducing a new gap.
- Extend gap penalty: A smaller penalty for continuing an existing gap.

Lastly, the actual comparison is typically performed as such:
<br>
<code>alignments = aligner.align(target, query)</code> # Stores a list-like object of all the best alignments; iterate through it to show all the alignments.

In [43]:
# Example pairwise alignment:
from Bio.Align import PairwiseAligner

reference = 'ATGCGTACCTGACTG' # Reference DNA

samples = [
    'ATGCGTACCTGACTG',   # perfect match
    'ATGCGTACGACTG',     # deletion
    'ATGCGTTCCTGACTG',   # mismatch
    'ATGCGTACC--ACTG',   # simulated insertion/deletion
]

aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 1
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -0.5

# This just shows the best alignments and score between reference and the perfect match in the samples list:
alignments = aligner.align(reference, samples[0])
for alignment in alignments:
    print(alignment)
    print(alignment.score)

print('--------------------------------------')

# Here are all of the best alignments between reference and all of the samples:
for i, sample in enumerate(samples):
    alignments = aligner.align(reference, sample)
    print(f'sample {i + 1}:')
    for alignment in alignments:
        print(alignment)
    print(f'score{alignments.score}')
    print('------------')

target            0 ATGCGTACCTGACTG 15
                  0 ||||||||||||||| 15
query             0 ATGCGTACCTGACTG 15

15.0
--------------------------------------
sample 1:
target            0 ATGCGTACCTGACTG 15
                  0 ||||||||||||||| 15
query             0 ATGCGTACCTGACTG 15

score15.0
------------
sample 2:
target            0 ATGCGTACCTGACTG 15
                  0 ||||||||--||||| 15
query             0 ATGCGTAC--GACTG 13

score10.5
------------
sample 3:
target            0 ATGCGTACCTGACTG 15
                  0 ||||||.|||||||| 15
query             0 ATGCGTTCCTGACTG 15

score13.0
------------
sample 4:
target            0 ATGCGTACCTGACTG 15
                  0 |||||||||--|||| 15
query             0 ATGCGTACC--ACTG 15

score11.0
------------


In [189]:
# This cell contains my custom function that returns a bunch of important information about the sequence, such as...
# Number of matches
# Location of matches
# Number of mismatches
# Location of mismatches
# Number of gap characters
# Number of gaps (or gap events)
# The locations of each gap (shown as lists of each gap's gap characters' positions)
def detect(reference, samples):
    for i, sample in enumerate(samples):
        alignments = aligner.align(reference, sample)
        for alignment in alignments:
            print(alignment)
            print(f'Score: {alignment.score}')

            aligned_str = str(alignment)
            lines = aligned_str.split('\n')

            target = lines[0].split()[2]
            query = lines[2].split()[2]

            n_matches = 0
            match_pos = []
            n_mismatches = 0
            mismatch_pos = []
            n_dashes = 0
            dash_pos = []
            n_gaps = 0
            gap_pos = []

            for j, (target_nuc, query_nuc) in enumerate(zip(target, query)):
                if (target_nuc == query_nuc):
                    n_matches += 1
                    match_pos.append(j + 1)
                elif (target_nuc != query_nuc) and (target_nuc != '-' and query_nuc != '-'):
                    n_mismatches += 1
                    mismatch_pos.append(j + 1)
                else:
                    n_dashes += 1
                    dash_pos.append(j + 1)

            print(f'Matches: {n_matches}\nMatch positions: {match_pos}\nMismatches: {n_mismatches}\nMismatch positions: {mismatch_pos}')

            if dash_pos:
                temp = [dash_pos[0]]
                gap_pos = []

                for i in range(1, len(dash_pos)):
                    if dash_pos[i] == dash_pos[i - 1] + 1:
                        temp.append(dash_pos[i])
                    else:
                        if len(temp) > 1:
                            gap_pos.append(temp)
                        temp = [dash_pos[i]]
                if len(temp) > 1:
                    gap_pos.append(temp)

                for gap in gap_pos:
                    n_gaps += 1

                print(f'Gap Characters: {n_dashes}\nGap Events: {n_gaps}\nGap positions: {gap_pos}')
            else:
                print('No gaps present')
            print('--------------------------------------')

detect(reference, samples)       

target            0 ATGCGTACCTGACTG 15
                  0 ||||||||||||||| 15
query             0 ATGCGTACCTGACTG 15

Score: 15.0
Matches: 15
Match positions: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Mismatches: 0
Mismatch positions: []
No gaps present
--------------------------------------
target            0 ATGCGTACCTGACTG 15
                  0 ||||||||--||||| 15
query             0 ATGCGTAC--GACTG 13

Score: 10.5
Matches: 13
Match positions: [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15]
Mismatches: 0
Mismatch positions: []
Gap Characters: 2
Gap Events: 1
Gap positions: [[9, 10]]
--------------------------------------
target            0 ATGCGTACCTGACTG 15
                  0 ||||||.|||||||| 15
query             0 ATGCGTTCCTGACTG 15

Score: 13.0
Matches: 14
Match positions: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
Mismatches: 1
Mismatch positions: [7]
No gaps present
--------------------------------------
target            0 ATGCGTACCTGACTG 15
                  0 

In [ ]:
list[0].id
CDS_1 = list[0].seq[60:1041]
print(CDS_1)
print()
list[1].id
CDS_2 = list[1].seq[0:939]
print(CDS_2)

detect(CDS_1, CDS_2)

In [173]:
# Testing cell
ex = aligner.align(reference, samples[1])

aligned_str = str(ex[0])
lines = aligned_str.split('\n')

target = lines[0].split()[2]
query = lines[2].split()[2]
print(target)
print(query)

n_matches = 0
match_pos = []
n_mismatches = 0
mismatch_pos = []
n_dashes = 0
dash_pos = []
n_gaps = 0
gap_pos = []

for i, (target_nuc, query_nuc) in enumerate(zip(target, query)):
    if (target_nuc == query_nuc):
        n_matches += 1
        match_pos.append(i + 1)
    elif (target_nuc != query_nuc) and (target_nuc != '-' and query_nuc != '-'):
        n_mismatches += 1
        mismatch_pos.append(i + 1)
    else:
        n_dashes += 1
        dash_pos.append(i + 1)

print(n_matches, match_pos, n_mismatches, mismatch_pos, n_dashes, dash_pos)


for i, dash_index in enumerate(dash_pos):
    dash_pos[i] = dash_index - 1

print(dash_pos)

temp = [dash_pos[0]]
gap_pos = []

for i in range(1, len(dash_pos)):
    if dash_pos[i] == dash_pos[i - 1] + 1:
        temp.append(dash_pos[i])
    else:
        if len(temp) > 1:
            gap_pos.append(temp)
        temp = [dash_pos[i]]

if len(temp) > 1:
    gap_pos.append(temp)

print(gap_pos)

for gap in gap_pos:
    n_gaps += 1

print(n_gaps)

ATGCGTACCTGACTG
ATGCGTAC--GACTG
13 [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15] 0 [] 2 [9, 10]
[8, 9]
[[8, 9]]
1


In [172]:
# Another testing cell
nums = [4, 5, 6, 1, 3, 4]
sequences = []
temp = [nums[0]]

for i in range(1, len(nums)):
    if nums[i] == nums[i - 1] + 1:
        temp.append(nums[i])
    else:
        if len(temp) > 1:
            sequences.append(temp)
        temp = [nums[i]]

if len(temp) > 1:
    sequences.append(temp)
    
print(sequences)

[[4, 5, 6], [3, 4]]
